In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import os
import openai
import getpass

from langchain_openai import OpenAIEmbeddings

API_KEY = ""
openai.api_key  = API_KEY
os.environ["OPENAI_API_KEY"] = API_KEY
embedding = OpenAIEmbeddings()

loader = PyPDFLoader("data_src.pdf")
pages = loader.load()
print(len(pages))
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)
docs = text_splitter.split_documents(pages)
print(len(docs))

from weaviate import Client, schema
import fitz  # PyMuPDF
from transformers import AutoTokenizer, AutoModel
from langchain.llms import OpenAI

client = Client("http://localhost:8080")


class_obj = {
  "class": "Document",
  "description": "A class to store documents.",
  "properties": [
    {
      "dataType": ["text"],
      "description": "The title of the document",
      "name": "title"
    },
    {
      "dataType": ["text"],
      "description": "The content of the document",
      "name": "content"
    }
  ],
  "vectorIndexType": "hnsw",
  "vectorizer": "none" 
}

client.schema.delete_all() 
client.schema.create_class(class_obj)

def insert_document(title, content, vector):
    document = {
        "title": title,
        "content": content
    }
    client.data_object.create(document, class_name="Document", vector=vector)


pdf_path = "data_src.pdf"
title = "data_src"  #

embeddings = []
for i in range(len(docs)):
    print(i)
    embeddings.append(embedding.embed_query(docs[i].page_content))
    insert_document(title, docs[i].page_content, embeddings[i])


query = """what are Recurrent neural networks,"""
query_embedding = embedding.embed_query(query)


k = 5 # Number of similar items to retrieve
vector_query = {
    "vector": query_embedding
}

results = client.query.get(
    "Document", 
    ["content"]  # Properties you want to retrieve
).with_near_vector(
    vector_query  # Use the correctly formatted query
).with_limit(k).do()


similar_result_queries = []
for result in results["data"]["Get"]["Document"]:
    print(result["content"])
    similar_result_queries.append(result["content"])